In [15]:
import pandas as pd

In [30]:
df = pd.read_csv("/home/aidl/Downloads/MLDS/Dataset/Train.csv")
df

,Unnamed: 0,text,genre,label,label_model,text_cleaned
0,0,"It starts with pain, followed by hate\nFueled ...",rock,9,LABEL_9,"It starts with pain, followed by hate\nFueled ..."
1,1,Freedom!\nAlone again again alone\nPatiently w...,rock,9,LABEL_9,Freedom!\nAlone again again alone\nPatiently w...
2,2,"Biting the hand that feeds you, lying to the v...",rock,9,LABEL_9,"Biting the hand that feeds you, lying to the v..."
3,3,You say you know just who I am\nBut you can't ...,rock,9,LABEL_9,You say you know just who I am\nBut you can't ...
4,4,My heart is beating faster can't control these...,rock,9,LABEL_9,My heart is beating faster can't control these...
...,...,...,...,...,...,...
290178,290178,I'm the best friend he's got I'd give him the ...,r&b,8,LABEL_8,I'm the best friend he's got I'd give him the ...
290179,290179,"Bad Boys Blue ""I Totally Miss You"" I did you w...",pop,7,LABEL_7,"Bad Boys Blue ""I Totally Miss You"" I did you w..."
290180,290180,Forgive me for the things That I never said to...,pop,7,LABEL_7,Forgive me for the things That I never said to...
290181,290181,The day they found a cure for AIDS The day the...,indie,4,LABEL_4,The day they found a cure for AIDS The day the...


In [17]:
df = df.dropna()
df.isnull().sum()

Unnamed: 0      0
text            0
genre           0
label           0
label_model     0
text_cleaned    0
dtype: int64

In [18]:
df.groupby('label').size()

label
0      1890
1      2213
2      8644
3      2240
4      8449
5     13545
6     20287
7    108694
8      2793
9    121392
dtype: int64

### Due to class Imbalance

In [19]:

def reduce_class_imbalance(df, label_column, max_samples=13000):
    adjusted_data = []
    for label, group in df.groupby(label_column):
        if len(group) > max_samples:
        
            adjusted_data.append(group.sample(n=max_samples, random_state=42))
        else:
            adjusted_data.append(group)
    balanced_df = pd.concat(adjusted_data).sample(frac=1, random_state=42)  
    return balanced_df

df = reduce_class_imbalance(df , label_column='label')
df.groupby('label').size()

label
0     1890
1     2213
2     8644
3     2240
4     8449
5    13000
6    13000
7    13000
8     2793
9    13000
dtype: int64

In [20]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AdamW
from transformers import get_scheduler
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm
import numpy as np

In [21]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        if not isinstance(text, str):
            raise ValueError(f"Expected a string for text at index {idx}, but got {type(text)}")

        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )

        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "label": torch.tensor(label, dtype=torch.long),
        }

In [22]:
texts = df["text_cleaned"]
labels = df["label"]     
num_classes = len(set(labels))

from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts, labels, test_size=0.2, stratify=labels, random_state=42
)

from transformers import  BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

train_dataset = CustomDataset(train_texts.tolist(), train_labels.tolist(), tokenizer)
val_dataset = CustomDataset(val_texts.tolist(), val_labels.tolist(), tokenizer)

from torch.utils.data import DataLoader
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

In [23]:
import torch
from tqdm import tqdm
from transformers import  BertForSequenceClassification, AdamW
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_classes)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [25]:
device

device(type='cuda', index=1)

In [26]:
model = model.to(device)

In [27]:
from transformers import AdamW
import torch
from tqdm import tqdm

optimizer = AdamW(model.parameters(), lr=5e-5)
epochs = 2

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader, desc=f"Training Epoch {epoch + 1}"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()


    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}, Training Loss: {avg_train_loss}")
    model.eval()  
    total_val_loss = 0
    correct_preds = 0
    total_preds = 0

    with torch.no_grad():  
        for batch in tqdm(val_loader, desc="Validating"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_val_loss += loss.item()
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
            correct_preds += (predictions == labels).sum().item()
            total_preds += labels.size(0)

    avg_val_loss = total_val_loss / len(val_loader)
    val_accuracy = correct_preds / total_preds
    print(f"Epoch {epoch + 1}, Validation Loss: {avg_val_loss}, Validation Accuracy: {val_accuracy}")

/home/aidl/.local/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Training Epoch 1:   0%|          | 0/7823 [00:00<?, ?it/s]

Training Epoch 1: 100%|██████████| 7823/7823 [1:13:48<00:00,  1.77it/s]


Epoch 1, Training Loss: 1.5619708537240171


Validating: 100%|██████████| 1956/1956 [06:40<00:00,  4.88it/s]


Epoch 1, Validation Loss: 1.4698190800464714, Validation Accuracy: 0.488111977502237


Training Epoch 2: 100%|██████████| 7823/7823 [1:14:18<00:00,  1.75it/s]


Epoch 2, Training Loss: 1.3132224413579983


Validating: 100%|██████████| 1956/1956 [06:40<00:00,  4.88it/s]

Epoch 2, Validation Loss: 1.4378631934066493, Validation Accuracy: 0.5130384762878691


In [29]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

save_path = "/home/aidl/Downloads/MLDS/Dataset/saved"

# 1. Save the model in `.safetensors` format
model.save_pretrained(save_path, safe_serialization=True)
tokenizer.save_pretrained(save_path)
print(f"Model and tokenizer saved in .safetensors format at: {save_path}")

# 2. Save the model weights in `.json` format
weights = model.state_dict()
weights_json = {k: v.tolist() for k, v in weights.items()}

import json
with open(f"{save_path}/model_weights.json", "w") as f:
    json.dump(weights_json, f)
print("Model weights saved as .json")

# 3. Save the model weights in `.txt` format (JSON serialized as text)
with open(f"{save_path}/model_weights.txt", "w") as f:
    f.write(json.dumps(weights_json))
print("Model weights saved as .txt")


Model and tokenizer saved in .safetensors format at: /home/aidl/Downloads/MLDS/Dataset/saved
Model weights saved as .json
Model weights saved as .txt


In [35]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import json

# Load the dataset to retrieve the original label column
data_path = "Train.csv"  # Replace with your actual dataset path
df = pd.read_csv(data_path)

# Initialize and fit the LabelEncoder on the original label column
label_encoder = LabelEncoder()
label_encoder.fit(df['label_model'])  # Assuming 'label_model' is the column you used for training

# Save the label mappings with typecasting
save_path = "/home/aidl/Downloads/MLDS/Dataset"
label_mapping = {str(key): int(value) for key, value in zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))}

# Write to JSON file
with open(f"{save_path}/label_encoder.json", "w") as f:
    json.dump(label_mapping, f)

print("Label mappings saved.")

Label mappings saved.


In [38]:
import torch
import transformers

# Check the version of PyTorch
print(f"PyTorch version: {torch.__version__}")

# Check the version of Transformers
print(f"Transformers version: {transformers.__version__}")

# Check the version of the tokenizer
from transformers import AutoTokenizer

# Example: Using 'bert-base-uncased', replace with your tokenizer if different
tokenizer_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(tokenizer_name)
print(f"Tokenizer '{tokenizer_name}' version: {tokenizer.version}")


PyTorch version: 2.6.0.dev20241106+cu118
Transformers version: 4.48.1


AttributeError: BertTokenizer has no attribute version